### This script will assume that the files you are moving around have the final status
### If you add lower status files to processed_files field, permissions will not work properly

In [ ]:
from dcicutils import ff_utils
from src.functions.notebook_functions import *
from src.functions.wfr import *

# get key from keypairs.json
my_key = get_key('')

# Use title to map the correct opf items
# move_title = 'HiC Processing Pipeline - Preliminary Files'
# move_title = "Repli-Seq Pipeline - Preliminary Files"
move_title = ''

# define the scope - which sets you want to process
set_url = '/search/?...'

# if True do the action, if false just report
action = False

# move other processed files to processed files field
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        print('ERROR: There are files in processed_files field, expected empty')
        return False
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], end= ': ')
            move_item = [i for i in opc if i['title'] == move_title]
            try:
                assert len(move_item) == 1
            except:
                print('ERROR: There are more than 1 opf set that has same title')
                return False
            try:
                assert move_item[0]['type'] == 'preliminary'
            except:
                print('ERROR: OPF set status should be preliminary')
                return False
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            # compare the status
            set_exp_status = resp['status']
            for a_file in new_pc:
                file_resp = ff_utils.get_metadata(a_file, con_key)
                file_status = file_resp['status']
                if file_status != set_exp_status:
                    print('ERROR: File status does not match the set, please use release script to update status for all related items')
                    return False
            if action:
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                print(len(new_pc), 'files moved to pf')
            else:
                print(len(new_pc), 'files will move to pf')
            return True
        else:
            return False


run_sets = ff_utils.search_metadata(set_url , key=my_key)
set_w_apf = 0
exp_w_apf = 0
counter = 0

if not action:
        print('\nTHIS IS JUST A TEST\n')   
        
print(len(run_sets), 'experiment sets in scope')
        
for a_set in run_sets:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_key)
    if res:
        set_w_apf += 1
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_key)
        if res_e:
            exp_w_apf += 1
    print()

print(set_w_apf, 'sets are updated')
print(exp_w_apf, 'exps are updated')